In [1]:
%%bash

head -n 10 Youtube01-Psy.csv

COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: kobyoshi02",1
LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,"Hey guys check out my new channel and our first vid THIS IS US THE  MONKEYS!!! I'm the monkey in the white shirt,please leave a like comment  and please subscribe!!!!",1
LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ   Check this out .﻿,1
LZQPQhLyRh9-wNRtlZDM90f1k0BrdVdJyN_YsaSwfxc,Jason Haddad,2013-11-26T02:55:11,"Hey, check out my new website!! This site is about kids stuff. kidsmediausa  . com",1
z13lfzdo5vmdi1cm123te5uz2mqig1brz04,ferleck ferles,2013-11-27T21:39

In [13]:
%%writefile mapper_psy.py
#!/usr/bin/env python
import csv
import sys
import re

def fun_datatime_mapper(datetimes):
    for datetime in datetimes:
        tokens_datetime = re.split(":", datetime)
        for token_datetime in tokens_datetime: 
            print(token_datetime + "\t1") 

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
datetimes=[row[2] for row in csvreader]  #get the "DATE" column
fun_datatime_mapper(datetimes)


Overwriting mapper_psy.py


In [14]:
%%writefile mapper_psy.py
#!/usr/bin/env python
import csv
import sys
import re
import string

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
datetimes=[row[2] for row in csvreader]  #get the "DATE" column
datetime_token_prefix=[]
date_reg='\d{4}[-/]\d{2}[-/]\d{2}[T]'
date_reg_exp=re.compile(date_reg)
for datetime in datetimes:
    datetime_tokens= re.split(r':', datetime)
    for datetime_token in datetime_tokens: 
#         print(datetime_token  + "\t1") 
#         if(string.find(datetime_token,'T')!=-1):
        if(re.match(date_reg_exp,datetime_token)):
            datetime_token_prefix.append(datetime_token)

for token in datetime_token_prefix:
     print(token  + "\t1") 


Overwriting mapper_psy.py


In [15]:
%%writefile reducer_psy.py
#!/usr/bin/env python

import sys
from collections import defaultdict

# test list
# datetime_input_pairs=[
#     '2013-11-07T06	1',
#     '2013-11-07T06	1',
#     '2014-11-03T20	1',
#     '2014-11-08T15	1',
#     '2014-11-08T15	1',
#     '2014-11-08T15	1',
#     '2014-11-06T24	1']

datetime_input_pairs = sys.stdin.readlines()
def fun_get_datetime_with_most_spam(datetime_input_pairs):
    datetime_accumulator = defaultdict(lambda: 0)
    datetime_with_most_spam_dict=defaultdict(lambda: 0)
    for row in datetime_input_pairs:
        datetime_key_value_pair = row.split("\t", 1)  #######row is string, split every row in input_pairs into 2(second parameter 1+1) parts

        if len(datetime_key_value_pair) != 2:
            continue

        word = datetime_key_value_pair[0]
        count = int(datetime_key_value_pair[1].strip()) #exception

        datetime_accumulator[word] = datetime_accumulator[word] + count

#     for (key, value) in  datetime_accumulator.items():
#         print(key + "\t" + str(value))
    
    datetime_with_most_spam=max(datetime_accumulator,key=datetime_accumulator.get)  #exception  ##just can find one
    datetime_with_most_spam_dict["datetime_with_most_spam_dict"]=datetime_with_most_spam
    return datetime_with_most_spam_dict
    

datetime_with_most_spam_dict=fun_get_datetime_with_most_spam(datetime_input_pairs)
for (key, value) in  datetime_with_most_spam_dict.items():
        print(key + "\t" + str(value))
    

Overwriting reducer_psy.py


In [16]:
%%bash
chmod a+x mapper_psy.py reducer_psy.py
cat Youtube01-Psy.csv | ./mapper_psy.py | ./reducer_psy.py | sort

datetime_with_most_spam_dict	2014-11-08T10


In [1]:
%%bash

rm -rf output

hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper_psy.py,reducer_psy.py \
-input Youtube01-Psy.csv \
-mapper ./mapper_psy.py \
-reducer ./reducer_psy.py \
-output output

Hadoop switched to standalone mode.


OpenJDK Server VM warning: You have loaded library /opt/hadoop-2.8.5/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
18/12/11 19:49:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/12/11 19:49:20 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/12/11 19:49:20 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/12/11 19:49:20 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
18/12/11 19:49:20 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/11 19:49:21 INFO mapreduce.JobSubmitter: number of splits:1
18/12/11 19:49:21 INFO mapreduce.JobSubmitter: Submitting token

In [21]:
%%bash
# cd output
# cat part-00000
cat output/part-00000

cat: output/part-00000: No such file or directory
